# Prompt Engineering Lifecycle

### Setup

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv()

True

### Log a trace

In [2]:
from app import langsmith_rag

question = "How do I set up tracing to LangSmith with @traceable?"
langsmith_rag(question)

USER_AGENT environment variable not set, consider setting it to identify your requests.


"To set up tracing to LangSmith with the @traceable decorator in Python, first ensure the LANGSMITH_TRACING environment variable is set to 'true' and the LANGSMITH_API_KEY is provided. Then, simply decorate any function you want to trace with @traceable. Remember to use the await keyword if you're wrapping synchronous functions to log traces correctly."

### Create a Dataset

Let's create a dataset to evaluate this particular step of our application

In [6]:
from langsmith import Client

example_dataset = [
    (
        "What is DRS in Formula 1?",
        """F1 Technical Regulations | Aerodynamics\n\nArticle 3.10: Aerodynamic Components\n\n3.10.8 Drag Reduction System (DRS)\n\nThe Drag Reduction System (DRS) is a driver-operated device that allows for an adjustable rear wing. The system is designed to aid overtaking. When activated, it opens a flap in the rear wing, which reduces aerodynamic drag and allows the car to achieve a higher top speed. The use of DRS is subject to specific conditions: it can only be activated in designated 'DRS zones' on the track during the race, and only when a car is detected to be less than one second behind the car in front at a specific detection point. The system is disabled immediately when the driver brakes. In practice sessions and qualifying, drivers may use it within the designated zones regardless of the gap to the car ahead.\n\nHome | Rules | Sporting Regulations | Technical Regulations | Contact\n\nF1 Explained: Overtaking Aids\nOne of the key innovations in modern Formula 1 is the Drag Reduction System. It was introduced in 2011 to promote more overtaking opportunities. The primary function is to reduce the 'dirty air' effect and give the following car a temporary speed advantage on the straights. Without it, passing can be notoriously difficult on certain circuits.""",
        "The Drag Reduction System (DRS) is a tool that helps with overtaking in F1. When activated by the driver in specific 'DRS zones' on the track, a flap on the rear wing opens. This reduces aerodynamic drag, giving the car a temporary speed boost. To use it in a race, a car must be less than one second behind another car at a designated detection point."
    ),
    (
        "How does the F1 points system work?",
        """FIA Formula 1 Sporting Regulations - Article 6: Championship Points\n\n6.4 Points for the Championship\nPoints are awarded to classified drivers and their teams for each Grand Prix according to the following scale:\n\n- 1st Place: 25 points\n- 2nd Place: 18 points\n- 3rd Place: 15 points\n- 4th Place: 12 points\n- 5th Place: 10 points\n- 6th Place: 8 points\n- 7th Place: 6 points\n- 8th Place: 4 points\n- 9th Place: 2 points\n- 10th Place: 1 point\n\n6.5 Fastest Lap Point\nAn additional point will be awarded to the driver who achieves the fastest valid lap time during the Grand Prix, provided they are also classified in the top 10 finishers. This point is also awarded to their team. If the driver with the fastest lap finishes outside the top 10, no fastest lap point is awarded for that race.\n\nRace Weekend Guide | Points and Standings | Sprint Race Rules\n\nUnderstanding the stakes in F1 means understanding the points. Every position in the top ten is crucial, with the winner taking a significant haul of 25 points. This structure ensures that consistency is rewarded, but victories are paramount for a championship campaign.""",
        "In a Formula 1 Grand Prix, points are awarded to the top 10 finishers as follows:\n\n- **1st**: 25 points\n- **2nd**: 18 points\n- **3rd**: 15 points\n- **4th**: 12 points\n- **5th**: 10 points\n- **6th**: 8 points\n- **7th**: 6 points\n- **8th**: 4 points\n- **9th**: 2 points\n- **10th**: 1 point\n\nAdditionally, one bonus point is given to the driver who sets the fastest lap, but only if they finish the race in the top 10."
    ),
    (
        "Can you explain the different tire compounds in F1?",
        """Pirelli Motorsport - The P Zero Range\n\nAt each Grand Prix weekend, Pirelli, the official tire supplier for Formula 1, provides three different slick tire compounds for the teams to use. These are chosen from a range of five compounds, labeled C1 (the hardest) to C5 (the softest).\n\nFor any given race, the three selected compounds are color-coded for easy identification:\n\n- **Hard Tire (White wall):** The most durable compound, designed for long stints. It offers less grip than the softer compounds but degrades much more slowly.\n- **Medium Tire (Yellow wall):** A balanced compound that offers a good compromise between performance and durability. It is a versatile tire often used as a key part of race strategy.\n- **Soft Tire (Red wall):** The least durable but highest-grip compound. It provides the most speed over a single lap, making it ideal for qualifying, but it wears out quickly during the race.\n\nIn addition to the slick tires, there are two options for wet weather conditions:\n\n- **Intermediate (Green wall):** For damp tracks or light rain.\n- **Full Wet (Blue wall):** For heavy rain and standing water.""",
        "Pirelli provides three slick tire compounds for each F1 race, color-coded for identification:\n\n- **Hard (White):** The most durable tire with the least grip, used for long race stints.\n- **Medium (Yellow):** A balanced tire offering a compromise between durability and speed.\n- **Soft (Red):** The fastest tire with the most grip, but it wears out very quickly. Ideal for qualifying.\n\nThere are also two wet weather tires: the **Intermediate (Green)** for damp conditions and the **Full Wet (Blue)** for heavy rain."
    )
]

client = Client()
dataset_name = "F1 questions"

# Create dataset
dataset = client.create_dataset(
    dataset_name=dataset_name, description="Sample questions about F1"
)

# Prepare inputs and outputs
inputs = [{"question": q, "context": c} for q, c, _ in example_dataset]
outputs = [{"output": o} for _, _, o in example_dataset]

# Create examples in the dataset
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    dataset_id=dataset.id,
)


{'example_ids': ['73566f62-6863-4677-9361-ed2da708530a',
  'c471d1fe-76aa-4718-9e9b-dd8e1953958f',
  'c84730b1-e361-45ab-a2d2-4ed2f1a85c6b'],
 'count': 3}

### Update our Application to use Prompt Hub

We're going to pretty much define the same RAG application as before - with one crucial improvement.

Instead of pulling our `RAG_PROMPT` from utils.py, we're going to connect to the Prompt Hub in LangSmith.

Let's add the code snippet that will pull down our prompt that we just iterated on!

In [10]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from langchain import hub
from langsmith.client import convert_prompt_to_openai_format
from openai import OpenAI
from typing import List
import nest_asyncio

MODEL_NAME = "gpt-4o-mini"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0

# TODO: Remove this hard-coded prompt and replace it with Prompt Hub

prompt = hub.pull("f1_simple_rag:f01615e0")


openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    # TODO: Let's use our prompt pulled from Prompt Hub instead of manually formatting here!

    formatted_prompt = prompt.invoke({"context":formatted_docs, "question": question})
    messages = convert_prompt_to_openai_format(formatted_prompt)["messages"]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [11]:
question = "How do I set up tracing to LangSmith with @traceable?"
langsmith_rag(question)

"To set up tracing to LangSmith using the `@traceable` decorator in Python, follow these steps:\n\n1. **Set Environment Variables**: \n   - Ensure you set the `LANGSMITH_TRACING` environment variable to `'true'`. This enables tracing.\n   - Set the `LANGSMITH_API_KEY` environment variable to your API key.\n\n2. **Install LangSmith**: Make sure you have the LangSmith SDK installed in your Python environment. You can use pip to install it if you haven't done so already.\n\n3. **Import the Traceable Decorator**: In your Python code, import the `traceable` decorator from the LangSmith SDK:\n\n   ```python\n   from langsmith import traceable\n   ```\n\n4. **Decorate Your Function**: Use the `@traceable` decorator to wrap any function you want to trace. Here’s an example:\n\n   ```python\n   @traceable\n   def my_function():\n       # Your function code here\n       pass\n   ```\n\n5. **Call Your Decorated Function**: When you call the decorated function, it will log traces to LangSmith:\n\n